In [35]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import seaborn as sns
import os

ModuleNotFoundError: No module named 'shapely'

In [2]:
!ls ../Datasets/

Code Legends		      job_listings.csv
_DS_Store		      labor_market_statistics.csv
employment_by_occupation.csv  lse_historical_data.csv
immigrant_statistics.csv      uk_bill_data.csv


In [29]:
# public job listings (see onet_code_legend.csv) and associated company info (see soc_code_legend.csv)
df1 = pd.read_csv('../Datasets/job_listings.csv', index_col=5, parse_dates=True) 
df1['delete_date'] = pd.to_datetime(df1['delete_date'])
df1.drop(columns=['url', 'last_updated', 'last_checked', 'ticker_start_date', 'ticker_end_date',
                 'title', 'company_name', 'stock_exchange_name', 'stock_exchange_country', 'stock_ticker'], inplace=True)
# ADD MORE FEATURES
df1['t_delta'] = (df1['delete_date'] - df1.index).dt.days # average difference between time posted and time deleted
df1['year'] = df1.index.year
df1['onet_fam'] = df1['onet_occupation_code'].str[:2]
df1.head()

/u/home/y/y1lo/project-miao/.conda/envs/pandas/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,city,region,zip,delete_date,onet_occupation_code,SOC_occupation_code,t_delta,year,onet_fam
created,,,,,,,,,
2014-12-10 12:36:33+00:00,Aberdeen,Scotland,AB25,2014-12-16 15:58:17+00:00,51-8013.00,8124,6.0,2014,51
2017-07-05 00:46:00+00:00,London,England,SW1A,2017-09-24 13:03:55+00:00,37-2021.00,6132,81.0,2017,37
2018-05-21 00:42:00+00:00,NaN,NaN,NaN,2018-06-12 09:56:00+00:00,99-9999.00,9999,22.0,2018,99
2017-11-02 19:51:00+00:00,Cambridge,England,NaN,2017-12-09 02:56:58+00:00,37-2021.00,6132,36.0,2017,37
2014-01-07 22:48:08+00:00,Aberdeen,Scotland,AB25,2014-04-08 19:27:57+00:00,15-1132.00,2136,90.0,2014,15


In [30]:
# UK employment data by occupation, status, sex (see soc_code_legend.csv for mapping)
df2 = pd.read_csv('../Datasets/employment_by_occupation.csv') 
df2.drop(columns=['soc_major', 'soc_minor', 'soc_unit', 'description', 'sex'], inplace=True)
df2.tail(20)

,year,soc_occupation_code,emp_type,fp_time,value
29932,2018,9274,self-employed,full,NaN
29933,2018,9274,self-employed,full,NaN
29934,2018,9274,self-employed,part,NaN
29935,2018,9274,self-employed,part,NaN
29936,2018,9275,employees,full,NaN
29937,2018,9275,employees,full,6830.0
29938,2018,9275,employees,part,16929.0
29939,2018,9275,employees,part,11429.0
29940,2018,9275,self-employed,full,NaN
29941,2018,9275,self-employed,full,NaN


In [34]:
sum(np.isnan(df2['value']))/df2.shape[0]

0.6203926282051282

In [27]:
sum(df2['value'].isnan)

AttributeError: 'Series' object has no attribute 'isnan'

# Assemble relevant data for training

In [15]:
df1.head()

,city,region,zip,delete_date,onet_occupation_code,SOC_occupation_code,t_delta,year,onet_fam
created,,,,,,,,,
2014-12-10 12:36:33+00:00,Aberdeen,Scotland,AB25,2014-12-16 15:58:17+00:00,51-8013.00,8124,6.0,2014,51
2017-07-05 00:46:00+00:00,London,England,SW1A,2017-09-24 13:03:55+00:00,37-2021.00,6132,81.0,2017,37
2018-05-21 00:42:00+00:00,NaN,NaN,NaN,2018-06-12 09:56:00+00:00,99-9999.00,9999,22.0,2018,99
2017-11-02 19:51:00+00:00,Cambridge,England,NaN,2017-12-09 02:56:58+00:00,37-2021.00,6132,36.0,2017,37
2014-01-07 22:48:08+00:00,Aberdeen,Scotland,AB25,2014-04-08 19:27:57+00:00,15-1132.00,2136,90.0,2014,15


In [28]:
df_agg = pd.merge(left=df1, right=df2, left_on='year', right_on='year')


MemoryError: Unable to allocate array with shape (8616568896,) and data type int64

In [21]:
df_agg.tail(20)

,city,region,zip,delete_date,onet_occupation_code,SOC_occupation_code,t_delta,year_x,onet_fam,year_y,soc_occupation_code,description,sex,emp_type,fp_time,value
139949036,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2016,5419,"Textiles, garments and related trades n.e.c.",male,self-employed,full,NaN
139949037,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2016,5419,"Textiles, garments and related trades n.e.c.",female,self-employed,full,NaN
139949038,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2016,5419,"Textiles, garments and related trades n.e.c.",male,self-employed,part,NaN
139949039,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2016,5419,"Textiles, garments and related trades n.e.c.",female,self-employed,part,NaN
139949040,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",male,employees,full,NaN
139949041,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",female,employees,full,NaN
139949042,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",male,employees,part,NaN
139949043,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",female,employees,part,NaN
139949044,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",male,self-employed,full,NaN
139949045,London,England,SW1A,2018-08-18 17:17:00+00:00,49-9093.00,5419,3.0,2018,49,2017,5419,"Textiles, garments and related trades n.e.c.",female,self-employed,full,NaN


In [9]:
# UK labor market stat, (see labor_market_stats_legend.csv for indicator codes and units)
df3 = pd.read_csv('../Datasets/labor_market_statistics.csv') 
df3.head()


,Year,Month,YBZY,YBZR,YBZQ,YBZT,YBZS,YBZV,YBZU,YBZW,...,LF3M,AIYW,LF3T,LF45,YCGN,AIYX,DZ36,LF3S,LF44,YCGM
0,1992,YEAR,38376.0,88044.0,24636.0,37656.0,50388.0,64752.0,118176.0,53424.0,...,71.5,6.0,67.7,60.8,6072.0,6.6,NaN,71.3,57.9,9264.0
1,1993,YEAR,38184.0,89076.0,23628.0,38556.0,50520.0,64968.0,119064.0,54096.0,...,70.8,5.2,67.5,60.8,6132.0,6.2,NaN,71.3,58.4,9360.0
2,1994,YEAR,38400.0,89784.0,22248.0,39024.0,50772.0,65016.0,119376.0,54360.0,...,71.5,5.2,69.3,61.2,5568.0,6.1,NaN,70.5,57.0,8592.0
3,1995,YEAR,38436.0,90072.0,21516.0,39492.0,50568.0,65604.0,120468.0,54864.0,...,71.4,5.2,69.8,60.3,5016.0,6.1,NaN,72.2,58.4,7908.0
4,1996,YEAR,39192.0,89976.0,20952.0,39780.0,50184.0,65772.0,121524.0,55752.0,...,72.1,5.7,70.0,61.6,4656.0,6.8,NaN,72.9,58.5,7392.0


In [11]:
df3.tail()

,Year,Month,YBZY,YBZR,YBZQ,YBZT,YBZS,YBZV,YBZU,YBZW,...,LF3M,AIYW,LF3T,LF45,YCGN,AIYX,DZ36,LF3S,LF44,YCGM
319,2019,MAY,4913.0,7831.0,1870.0,3642.0,4189.0,5879.0,11143.0,5264.0,...,78.2,12.2,74.4,71.0,146.0,13.6,NaN,76.9,67.5,268.0
320,2019,JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,2019,JUL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,2019,Q1,19680.0,31204.0,7424.0,14488.0,16716.0,23440.0,44608.0,21168.0,...,79.1,11.2,73.9,71.2,528.0,12.8,121.2,76.5,67.4,1012.0
323,2019,Q2,19652.0,31324.0,7480.0,14568.0,16756.0,23516.0,44572.0,21056.0,...,78.2,12.2,74.4,71.0,584.0,13.6,124.0,76.9,67.5,1072.0


In [6]:
# UK citizenship applications by country of nationality (se immigrant_legend.csv)
df4 = pd.read_csv('Table Headers/dataset_4.csv') 
df4.head()

,Year,Quarter,AFN,AFS,AFN.1,ACS,ACE,AEA,ASO,ASE,...,ERO-VTC,ACS-VNZ,ASE-VTN,OTH-VIB,AMN-VIU,OCE-WAF,AFN-WSH,MIE-YMN,AFS-ZMB,AFS-ZBW
0,2007,Q1,"1,396","20,485","1,391","3,337","4,625","1,747","13,305","6,026",...,0,54,101,0,0,0,0,102,248,"2,296"
1,2007,Q2,714,"8,163",864,"1,598","2,041",879,"7,054","2,171",...,0,36,76,0,0,0,0,76,102,891
2,2007,Q3,739,"8,648",774,"1,800","1,845",847,"6,838","2,142",...,0,33,68,0,0,0,0,69,108,"1,183"
3,2007,Q4,745,"9,087",672,"1,680","1,824",945,"6,096","1,778",...,0,21,81,0,0,0,0,74,111,"1,507"
4,2008,Q1,"1,019","13,398",804,"2,293","1,874",978,"8,536","2,079",...,0,36,72,0,0,0,1,115,151,"2,230"


In [15]:
# historical London Stock Exchange company fundamental data
df5 = pd.read_csv('../Datasets/lse_historical_data.csv') 
df5.drop(columns='ticker', inplace=True)
df5['list_date'] = pd.to_datetime(df5['list_date'], errors='coerce')
df5.head(20)

,year,month,list_date,company_name,icb_industry,icb_supersector,country_of_incorporation,market_cap,eps,p/e,p/s
0,2017,11,2006-08-02,1PM PLC,Financial Services,Financials,United Kingdom,40.036935,NaN,NaN,NaN
1,2017,11,2009-02-02,1SPATIAL PLC,Industrial Goods & Services,Industrials,United Kingdom,29.467662,NaN,NaN,NaN
2,2017,11,2005-04-15,21ST CENTURY TECHNOLOGY PLC,Industrial Goods & Services,Industrials,United Kingdom,2.564093,NaN,NaN,NaN
3,2017,11,1994-07-18,3I GROUP PLC,Financial Services,Financials,United Kingdom,8765.510388,NaN,NaN,NaN
4,2017,11,2007-03-13,3I INFRASTRUCTURE PLC,Financial Services,Financials,Jersey,2025.382649,NaN,NaN,NaN
5,2017,11,2014-02-18,4D PHARMA PLC,Healthcare,Health Care,United Kingdom,221.041717,NaN,NaN,NaN
6,2017,11,1953-03-13,4IMPRINT GROUP PLC,Media,Consumer Services,United Kingdom,512.560923,NaN,NaN,NaN
7,2017,11,2011-07-14,600 GROUP PLC,Industrial Goods & Services,Industrials,United Kingdom,16.381134,NaN,NaN,NaN
8,2017,11,2014-06-10,7DIGITAL GROUP PLC,Retail,Consumer Services,United Kingdom,9.447723,NaN,NaN,NaN
9,2017,11,2005-10-04,888 HOLDINGS PLC,Travel & Leisure,Consumer Services,Gibraltar,933.331404,NaN,NaN,NaN


In [8]:
# UK Public General Acts, content and size of bill
df6 = pd.read_csv('Table Headers/dataset_6.csv', index_col=1, parse_dates=True) 
df6.drop(columns=['link_to_act', 'characters_in_act', 'words_in_act'], inplace=True)
df6.head()

,act_name,amendment,act_text
date_of_act,,,
2019-02-12,Finance Act 2019,False,Most Gracious Sovereign WE; Your Majestys most...
2019-02-12,Voyeurism (Offences) Act 2019,False,Be it enacted by the Queens most Excellent Maj...
2019-02-12,Counter-Terrorism and Border Security Act 2019,False,Be it enacted by the Queens most Excellent Maj...
2019-02-12,Tenant Fees Act 2019,False,Be it enacted by the Queens most Excellent Maj...
2019-02-12,Crime (Overseas Production Orders) Act 2019,False,Be it enacted by the Queens most Excellent Maj...
